<a href="https://colab.research.google.com/github/kelvinfoo123/Natural-Language-Processing/blob/main/Topic_Modelling_on_Trip_Advisor_Reviews_(LDA_and_Gensim).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt 

import re 
import gensim 
import spacy 
from gensim.models import CoherenceModel 
from gensim.utils import simple_preprocess
import gensim.corpora as corpora

import nltk
from nltk.corpus import stopwords 

In [5]:
nltk.download('stopwords')
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use', 'not', 'would', 'say', 'could', '_', 'be', 'know', 
                   'good', 'go', 'get', 'do', 'done', 'try', 'many', 'some', 'nice', 'thank', 'think', 'see',
                   'rather', 'easy', 'easily', 'lot', 'lack', 'make', 'want', 'seem', 'run', 'need', 'even', 
                   'right', 'line', 'even', 'also', 'may', 'take', 'come'])

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [7]:
df = pd.read_csv("review.csv")
df.head()

,Review,Rating
0,nice hotel expensive parking got good deal sta...,4
1,ok nothing special charge diamond member hilto...,2
2,nice rooms not 4* experience hotel monaco seat...,3
3,"unique, great stay, wonderful time hotel monac...",5
4,"great stay great stay, went seahawk game aweso...",5


In [9]:
print("Number of reviews: ", df.shape[0])

Number of reviews:  20491


In [10]:
# Just consider 5000 reviews due to time constraints 
df = df[0:5000]

## **Tokenize and clean sentences**

In [11]:
def tokenize(sentence): 
  for sent in sentence: 
    sent = re.sub('\S*@\S*\s?', '', sent) # remove emails 
    sent = re.sub('\s+', '', sent) # remove newline chars 
    sent = re.sub("\'", '', sent) # remove single quotes 
    sent = gensim.utils.simple_preprocess(str(sent), deacc = True) # Convert into a list of tokens 
    yield(sent)

In [16]:
data = df.Review.values.tolist()
data_words = list(tokenize(data))
print(data_words[:3]) # Tokens for first 3 reviews 

[['checkquickeasy', 'parkingnight'], ['gotkidding', 'hours', 'bedscomfortable', 'notgoodac', 'heatcontrol', 'thisnot'], ['niceroomsnot', 'level', 'missed', 'nightgot', 'gotroomnightno', 'stdropdesk', 'askedwakeup', 'calledlater']]


## **Lemmatize and build bigram, trigram models**

- min_count ignore all words and bigrams with total collected count lower than this.
- threshold represents a threshold for forming the phrases (higher means fewer phrases). A phrase of words a and b is accepted if (cnt(a, b) - min_count) * N / (cnt(a) * cnt(b)) > threshold, where N is the total vocabulary size.

In [18]:
bigram = gensim.models.Phrases(data_words, min_count = 5, threshold = 100)
trigram = gensim.models.Phrases(bigram[data_words], threshold = 100)

/usr/local/lib/python3.8/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [21]:
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [28]:
nlp = spacy.load('en_core_web_sm')

def process(text, stop_words = stop_words, allowed_posttags = ['NOUN', 'ADJ', 'VERB', 'ADV']): 
  text = [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in text]
  text = [bigram_mod[doc] for doc in text]
  text = [trigram_mod[bigram_mod[doc]] for doc in text]

  text_out = []
  for sent in text: 
    doc = nlp(" ".join(sent))
    text_out.append([token.lemma_ for token in doc if token.pos_ in allowed_posttags])

  text_out = [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in text_out]
  return text_out 

In [29]:
data_ready = process(data_words)

## **Latent Dirichlet Allocation**

In [30]:
# Create dictionary 
dictionary = corpora.Dictionary(data_ready)

In [32]:
# Create term document frequency 
corpus = [dictionary.doc2bow(text) for text in data_ready]

In [33]:
# Build LDA model 
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=dictionary,
                                           num_topics=4, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=10,
                                           passes=10,
                                           alpha='symmetric',
                                           iterations=100,
                                           per_word_topics=True)

In [34]:
from pprint import pprint 
pprint(lda_model.print_topics())

[(0,
  '0.024*"star" + 0.019*"room" + 0.010*"starhotel" + 0.008*"spend" + '
  '0.008*"lovedplace" + 0.008*"min" + 0.008*"time" + 0.006*"highlyrecommend" + '
  '0.005*"morning" + 0.005*"travel"'),
 (1,
  '0.013*"hotel" + 0.012*"love" + 0.011*"rd" + 0.009*"com" + '
  '0.009*"stafffriendly" + 0.009*"check" + 0.007*"defintelystay" + '
  '0.006*"nightstay" + 0.005*"excellent" + 0.005*"fun"'),
 (2,
  '0.016*"euro" + 0.008*"minutewalkaway" + 0.007*"enjoy" + 0.007*"pay" + '
  '0.007*"euros" + 0.007*"arrive" + 0.006*"nicehotel" + 0.006*"breakfast" + '
  '0.005*"roomservice" + 0.005*"hour"'),
 (3,
  '0.043*"night" + 0.029*"stay" + 0.013*"perfectlocation" + 0.012*"minute" + '
  '0.012*"day" + 0.009*"wonderful" + 0.009*"fabulous" + 0.009*"great" + '
  '0.009*"pm" + 0.008*"definitelystay"')]


## **Visualization of topic**

In [39]:
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis 
pyLDAvis.enable_notebook()

In [40]:
vis = gensimvis.prepare(lda_model, corpus, dictionary = lda_model.id2word)
vis

/usr/local/lib/python3.8/dist-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3     -0.137507  0.054655       1        1  27.148283
0      0.017352 -0.146445       2        1  25.705281
2     -0.007586  0.012332       3        1  25.260917
1      0.127741  0.079459       4        1  21.885518, topic_info=               Term        Freq       Total Category  logprob  loglift
30            night  182.000000  182.000000  Default  30.0000  30.0000
59             stay  121.000000  121.000000  Default  29.0000  29.0000
382            star   95.000000   95.000000  Default  28.0000  28.0000
19             room   77.000000   77.000000  Default  27.0000  27.0000
9067           euro   62.000000   62.000000  Default  26.0000  26.0000
...             ...         ...         ...      ...      ...      ...
9666     nightswish   15.499488   16.776634   Topic4  -5.3877   1.4402
9667  roomntperfect   15.499488   16.776634   Topic4  -5.3877   1.4402
5154       roomhuge   14.253814   15.384270   Topic4  -5.4715   1.4430
9633       easyfoot   13.641262   14.737480   Topic4  -5.5154   1.4421
9658   smokerchecke   14.758324   16.906339   Topic4  -5.4367   1.3835

[180 rows x 6 columns], token_table=      Topic      Freq             Term
term                                  
1163      3  0.972992            adult
9611      1  0.940630         ammonday
9566      3  0.900691      aohgoodness
9627      1  0.925825        apartment
9588      1  0.951310          armoire
...     ...       ...              ...
9660      2  0.875672  wontreturnhotel
9660      3  0.051510  wontreturnhotel
9660      4  0.051510  wontreturnhotel
9294      4  0.938682      worthafford
205       2  0.915722             year

[181 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 1, 3, 2])